### Business Recommender

#### In this notebook we are considering businesses coming under food category and will provide suggestions base on that



##### We will start by importing required libraries for data processing

In [ ]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

#### We will extract information of all the neighborhoods in Manhattan borough
newyork_data.json file is geojson file which contains required attributes we will fetch below mentioned attributes from the file

1 Borough : In order to apply filter for selecting only Manhattan borough

2 Neighborhood : Name of neighborhood

3 Latitude : To pull Venues for neighborhood from Foursquare API

4 Longitude: To pull Venues for neighborhood from Foursquare API




In [2]:
!wget -q -O 'newyork_data.json' https://ibm.box.com/shared/static/fbpwbovar7lf8p5sgddm06cgipa2rxpe.json
print('Data downloaded!')

with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)
neighborhoods_data = newyork_data['features']

# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)


Data downloaded!


###### in order to fetch venues for each and every neighborhood we need to call Explore endpoint of Foursquare api 
below mentioned function will return dataframe containing venues for 1 neighbourhod at a time. 
section variable restrict the venue to food category.

In [3]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    section = 'food'
    LIMIT =200
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&section={}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng,
            section,
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            id,
            name, 
            lat, 
            lng,              
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude',         
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [4]:
CLIENT_ID = 'RFVDSYOWWO2LFEB0Q4T3EYEPU2HFGSIYE35LMXNXIZT0H302' # your Foursquare ID
CLIENT_SECRET = 'E5BWKVHOKS4QCACDRQLAFVWRHUB0T3U5NGZXFF0MRB2HF13T' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: RFVDSYOWWO2LFEB0Q4T3EYEPU2HFGSIYE35LMXNXIZT0H302
CLIENT_SECRET:E5BWKVHOKS4QCACDRQLAFVWRHUB0T3U5NGZXFF0MRB2HF13T


In [46]:
# type your answer here
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],latitudes=manhattan_data['Latitude'],longitudes=manhattan_data['Longitude'])

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


Since there is a limit on how many calls we can make to Foursquare APIs we have stored above result in local file and for subsequent trials we use below mentioned code to get the dataset instead of calling APIs

In [6]:

import sys
import types
import pandas as pd
from ibm_botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share your notebook.
client_248025d529414dfa818c41acdf43a4e9 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='9KlRQmZX73IwxYmE7s7f_BIHaL3ZeNc-KOgwwa4yc6gf',
    ibm_auth_endpoint="https://iam.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_248025d529414dfa818c41acdf43a4e9.get_object(Bucket='advanceddatasciencecapstone-donotdelete-pr-ycclchulzqf7w0',Key='manhattan_venues.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

manhattan_venues = pd.read_csv(body)
manhattan_venues.drop(['Unnamed: 0'], axis=1)

In order to identify top food business categories we will first count no of categories for each neighbourhood with help of
one hot encoding technique and then group rows by neighbourhood.

In [8]:
manhattan_venues.groupby('Neighborhood').count()

# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped


,Neighborhood,Afghan Restaurant,African Restaurant,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Asian Restaurant,Australian Restaurant,Austrian Restaurant,BBQ Joint,Bagel Shop,Bakery,Belgian Restaurant,Bistro,Brazilian Restaurant,Breakfast Spot,Buffet,Burger Joint,Burrito Place,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Chinese Restaurant,Creperie,Cuban Restaurant,Czech Restaurant,Deli / Bodega,Dim Sum Restaurant,Diner,Donut Shop,Dosa Place,Dumpling Restaurant,Eastern European Restaurant,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,Food,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Gastropub,German Restaurant,Gluten-free Restaurant,Greek Restaurant,Hawaiian Restaurant,Hot Dog Joint,Hotpot Restaurant,Indian Restaurant,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jewish Restaurant,Kebab Restaurant,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Lebanese Restaurant,Mac & Cheese Joint,Malay Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Molecular Gastronomy Restaurant,Moroccan Restaurant,New American Restaurant,Noodle House,North Indian Restaurant,Paella Restaurant,Pakistani Restaurant,Peking Duck Restaurant,Persian Restaurant,Peruvian Restaurant,Pet Café,Pizza Place,Poke Place,Portuguese Restaurant,Ramen Restaurant,Restaurant,Russian Restaurant,Salad Place,Sandwich Place,Scandinavian Restaurant,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Snack Place,Soba Restaurant,Soup Place,South American Restaurant,South Indian Restaurant,Southern / Soul Food Restaurant,Spanish Restaurant,Sri Lankan Restaurant,Steakhouse,Sushi Restaurant,Swiss Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Thai Restaurant,Theme Restaurant,Tibetan Restaurant,Tonkatsu Restaurant,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Vietnamese Restaurant,Wings Joint
0,Battery Park City,0.00,0.00000,0.055556,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556,0.000000,0.055556,0.000000,0.027778,0.000000,0.000000,0.00,0.083333,0.027778,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.055556,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.055556,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.027778,0.00,0.027778,0.000000,0.000000,0.027778,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.027778,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.027778,0.027778,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000,0.111111,0.000000,0.00,0.000000,0.027778,0.00,0.027778,0.055556,0.00,0.055556,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.027778,0.027778,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000
1,Carnegie Hill,0.00,0.00000,0.014286,0.000000,0.014286,0.000000,0.000000,0.000000,0.000000,0.014286,0.071429,0.000000,0.014286,0.014286,0.014286,0.00,0.000000,0.000000,0.000000,0.085714,0.000000,0.000000,0.00,0.000000,0.000000,0.014286,0.000000,0.000000,0.00,0.028571,0.00,0.028571,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.014286,0.014286,0.014286,0.000000,0.000000,0.00,0.028571,0.042857,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.014286,0.00,0.028571,0.000000,0.000000,0.071429,0.000000,0.042857,0.000000,0.000000,0.014286,0.014286,0.000000,0.000000,0.00,0.000000,0.014286,0.042857,0.000000,0.000000,0.000000,0.00,0.028571,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000,0.114286,0.000000,0.00,0.014286,0.028571,0.00,0.014286,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00000

Now we will sort the categories based on number of venues in that categories for every neighbourhood

In [9]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [16]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)
neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Pizza Place,Burger Joint,Italian Restaurant,BBQ Joint,Donut Shop,Bakery,Seafood Restaurant,Sandwich Place,Chinese Restaurant,American Restaurant
1,Carnegie Hill,Pizza Place,Café,Bakery,Italian Restaurant,Japanese Restaurant,Mexican Restaurant,French Restaurant,Sushi Restaurant,Diner,Deli / Bodega
2,Central Harlem,Fried Chicken Joint,Deli / Bodega,Southern / Soul Food Restaurant,Seafood Restaurant,African Restaurant,Pizza Place,Chinese Restaurant,Caribbean Restaurant,French Restaurant,BBQ Joint
3,Chelsea,Bakery,Italian Restaurant,American Restaurant,Café,Japanese Restaurant,Tapas Restaurant,Mexican Restaurant,Seafood Restaurant,Pizza Place,Breakfast Spot
4,Chinatown,Chinese Restaurant,Bakery,Dim Sum Restaurant,Vietnamese Restaurant,Dumpling Restaurant,Café,Mexican Restaurant,Hotpot Restaurant,Sandwich Place,American Restaurant
5,Civic Center,Italian Restaurant,Sandwich Place,American Restaurant,French Restaurant,Bakery,Café,Mexican Restaurant,Diner,Pizza Place,Korean Restaurant
6,Clinton,American Restaurant,Italian Restaurant,Deli / Bodega,Sandwich Place,Chinese Restaurant,Mexican Restaurant,Pizza Place,Restaurant,Café,New American Restaurant
7,East Harlem,Mexican Restaurant,Pizza Place,Deli / Bodega,Bakery,Latin American Restaurant,Fast Food Restaurant,Thai Restaurant,Burger Joint,Steakhouse,Spanish Restaurant
8,East Village,Pizza Place,Mexican Restaurant,Japanese Restaurant,Café,Vietnamese Restaurant,Ramen Restaurant,Italian Restaurant,Chinese Restaurant,French Restaurant,Vegetarian / Vegan Restaurant
9,Financial District,Sandwich Place,Italian Restaurant,American Restaurant,Mexican Restaurant,Café,Food Truck,Steakhouse,Deli / Bodega,Pizza Place,Salad Place


In [11]:
neighborhoods_venues_sorted.shape

(40, 11)

Now dataframe can be utilized to run KMeans algorithm on it in order to identify clusters

In [62]:
# set number of clusters
kclusters = 11

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:40]

array([ 1,  0,  8,  0,  0,  1,  1,  5,  0,  1,  3,  9,  3,  5,  1,  5,  9,
        1,  0,  0,  0,  5,  4,  1,  7, 10,  1,  9,  6,  3,  2,  9,  1, 10,
        1,  3,  3,  5,  3,  9], dtype=int32)

In order to visualize how clusters are distributed on map we will join the lables with original data which has longitude and latitude information and plot them on map using different color coding 

In [63]:
manhattan_merged = manhattan_data

# add clustering labels
manhattan_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

In [64]:
address = 'Manhattan, NY'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [74]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Marble Hill,Sandwich Place,Deli / Bodega,Restaurant,American Restaurant,Pizza Place,Diner,Donut Shop,Seafood Restaurant,Steakhouse,English Restaurant
5,Manhattanville,Deli / Bodega,Chinese Restaurant,Seafood Restaurant,Mexican Restaurant,Italian Restaurant,Sandwich Place,Fried Chicken Joint,Sushi Restaurant,Diner,Cuban Restaurant
6,Central Harlem,Fried Chicken Joint,Deli / Bodega,Southern / Soul Food Restaurant,Seafood Restaurant,African Restaurant,Pizza Place,Chinese Restaurant,Caribbean Restaurant,French Restaurant,BBQ Joint
9,Yorkville,Pizza Place,Italian Restaurant,Deli / Bodega,Japanese Restaurant,Thai Restaurant,Sushi Restaurant,Sandwich Place,Mexican Restaurant,Bakery,Bagel Shop
14,Clinton,American Restaurant,Italian Restaurant,Deli / Bodega,Sandwich Place,Chinese Restaurant,Mexican Restaurant,Pizza Place,Restaurant,Café,New American Restaurant
17,Chelsea,Bakery,Italian Restaurant,American Restaurant,Café,Japanese Restaurant,Tapas Restaurant,Mexican Restaurant,Seafood Restaurant,Pizza Place,Breakfast Spot
23,Soho,Italian Restaurant,French Restaurant,Café,American Restaurant,Bakery,Mediterranean Restaurant,Sandwich Place,Pizza Place,Mexican Restaurant,Seafood Restaurant
26,Morningside Heights,Deli / Bodega,Pizza Place,Café,Burger Joint,American Restaurant,Food Truck,Sandwich Place,Korean Restaurant,Chinese Restaurant,New American Restaurant
32,Civic Center,Italian Restaurant,Sandwich Place,American Restaurant,French Restaurant,Bakery,Café,Mexican Restaurant,Diner,Pizza Place,Korean Restaurant
34,Sutton Place,Pizza Place,Indian Restaurant,Italian Restaurant,Bagel Shop,American Restaurant,Mexican Restaurant,Burger Joint,Japanese Restaurant,Asian Restaurant,Mediterranean Restaurant
